In [1]:
import sys
sys.path.insert(0, '../')
import ccob_qe_analysis_BOT as ana
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import numpy as np
from lsst.obs.lsst.cameraTransforms import LsstCameraTransforms
from lsst.obs.lsst import LsstCamMapper as camMapper

In [3]:
camera = camMapper._makeCamera()
lct = LsstCameraTransforms(camera)
basedir1 = '/gpfs/slac/lsst/fs3/g/data/jobHarness/jh_stage-test/LCA-10134_Cryostat/LCA-10134_Cryostat-0001/'
basedir2 = '/gpfs/slac/lsst/fs3/g/data/jobHarness/jh_stage/LCA-10134_Cryostat/LCA-10134_Cryostat-0001/'

path_to_data = {'R01': basedir1 + '6848D/BOT_acq/v0/48087/',
                'R02': basedir1 + '6849D/BOT_acq/v0/48093/',
                'R10': None,
                'R11': basedir1 + '6851D/BOT_acq/v0/48108/',
                'R12': basedir1 + '6852D/BOT_acq/v0/48113/',
                'R20': basedir1 + '6853D/BOT_acq/v0/48118/',
                'R21': None,
                'R22': basedir2 + '11974/BOT_acq/v0/93868/',
                'R30': basedir1 + '6843D/BOT_acq/v0/48047/'
               }
path_to_beam = '/home/combet/tmp_9rafts/60x60/'
outdir_figs = '/home/combet/tmp_9rafts/QE_results/'
outdir_fits = '/gpfs/slac/lsst/fs1/u/combet/DATA/CCOB_QE/'
led_beam = 'nm960'
led = 'nm960'

model_ccdid = 'R22_S11'
beam_model = ana.load_beam_model(path_to_beam, led_name=led_beam, ref_amp=5, ccdid=model_ccdid)
delta_x, delta_y = ana.compute_offsets(beam_model, lct, ccdid=model_ccdid, ref_pix_x=2304, ref_pix_y=3003)

raft_list = ['R12','R20', 'R22','R30']
#raft_list = ['R01','R02','R11','R12','R20', 'R22','R30']
#raft_list = ['R30']
sensor_list = ['S00','S01','S02','S10','S11','S12','S20','S21','S22']
#sensor_list = ['S21','S22']

ccdid_list = sorted([raft+'_'+sensor for sensor in sensor_list for raft in raft_list])

for data_ccdid in ccdid_list[0:1]:
    gainfile = '/gpfs/slac/lsst/fs3/g/data/jobHarness/jh_stage-test/LCA-10134_Cryostat/LCA-10134_Cryostat-0001/6801D/fe55_analysis_BOT/v0/47706/'+data_ccdid+'_6801D_eotest_results.fits'

    raft = data_ccdid.split('_')[0]
    data = ana.qe_data.CcobQeData(data_ccdid, led, path_to_data[raft], gainfile, biasfile=None)
    data.find_dir()
    
    for pos in data.pos_list:
        data.make_avg_mosaic_at_pos(pos, '/home/combet/tmp_9rafts/')
        mosaic = data.data[pos]['mosaic']
        bbox = ana.define_model_bbox(beam_model, mosaic, data.lct, pos, delta_x, delta_y)

        xarr = np.linspace(bbox[0],bbox[1],mosaic.shape[0])
        yarr = np.linspace(bbox[2],bbox[3],mosaic.shape[1])
        tmp = beam_model.beam_image['f_interp'](xarr, yarr) # interp works in cam coordinates
        tmp = np.flip(np.flip(tmp,axis=0),axis=1) # invert the model in x and in y 
        model_eotestDef = np.flip(tmp.T, axis=1) # beam model following EOTest convention
        model_normalised = model_eotestDef/np.max(model_eotestDef.flatten())
        qe = mosaic/model_normalised
#        qe = mosaic/model_eotestDef
        qe_norm = qe / np.mean(qe.flatten())

        outfile = outdir_fits+'fits/QE_'+data_ccdid+'_'+led+'_'+pos+'.fits'
        amp_coord = data.data[pos]['amp_coord']
        ana.make_fits(qe,amp_coord, outfile, data.template_file)

#        figfile = outdir_figs+'figs/QE_'+data_ccdid+'_'+led+'_'+pos+'_from_'+led_beam+'_norm.png'
        figfile = outdir_figs+'figs/QE_'+data_ccdid+'_'+led+'_'+pos+'_norm.png'
        ana.plot_results(qe_norm, model_normalised, mosaic, data_ccdid, data.lct, pos, delta_x, delta_y,figfile)



Averaging the following files:

['/gpfs/slac/lsst/fs3/g/data/jobHarness/jh_stage-test/LCA-10134_Cryostat/LCA-10134_Cryostat-0001/6852D/BOT_acq/v0/48113/ccob_nm960_-159.175_-32.325_006/MC_C_20191028_005273_R12_S00.fits']
3.1496989966555202 14.95983277591975 37.81030100334448 25.080167224080252
40.96 40.040000000000006
(4004, 4096)
40.040000000000006
40.959999999999994
Averaging the following files:

['/gpfs/slac/lsst/fs3/g/data/jobHarness/jh_stage-test/LCA-10134_Cryostat/LCA-10134_Cryostat-0001/6852D/BOT_acq/v0/48113/ccob_nm960_-159.175_-52.805_000/MC_C_20191028_005267_R12_S00.fits']


../ccob_qe_data.py:86: ResourceWarning: unclosed file <_io.FileIO name='/home/combet/tmp_9rafts/tmp.fits' mode='rb' closefd=True>
  mosaic, amp_coord = u.make_ccd_2d_array(tmp_file, gains=self.gains, biasfile=self.biasfile)


23.629698996655517 14.95983277591975 17.330301003344484 25.080167224080252
40.96 40.040000000000006
(4004, 4096)
40.040000000000006
40.96
Averaging the following files:

['/gpfs/slac/lsst/fs3/g/data/jobHarness/jh_stage-test/LCA-10134_Cryostat/LCA-10134_Cryostat-0001/6852D/BOT_acq/v0/48113/ccob_nm960_-169.185_-42.565_012/MC_C_20191028_005279_R12_S00.fits']


../ccob_qe_data.py:86: ResourceWarning: unclosed file <_io.FileIO name='/home/combet/tmp_9rafts/tmp.fits' mode='rb' closefd=True>
  mosaic, amp_coord = u.make_ccd_2d_array(tmp_file, gains=self.gains, biasfile=self.biasfile)


13.389698996655516 24.96983277591973 27.570301003344486 15.070167224080269
40.96 40.04
(4004, 4096)
40.04
40.96000000000001
Averaging the following files:

['/gpfs/slac/lsst/fs3/g/data/jobHarness/jh_stage-test/LCA-10134_Cryostat/LCA-10134_Cryostat-0001/6852D/BOT_acq/v0/48113/ccob_nm960_-179.195_-32.325_024/MC_C_20191028_005291_R12_S00.fits']


../ccob_qe_data.py:86: ResourceWarning: unclosed file <_io.FileIO name='/home/combet/tmp_9rafts/tmp.fits' mode='rb' closefd=True>
  mosaic, amp_coord = u.make_ccd_2d_array(tmp_file, gains=self.gains, biasfile=self.biasfile)


3.1496989966555202 34.97983277591973 37.81030100334448 5.060167224080269
40.96 40.04
(4004, 4096)
40.04
40.959999999999994
Averaging the following files:

['/gpfs/slac/lsst/fs3/g/data/jobHarness/jh_stage-test/LCA-10134_Cryostat/LCA-10134_Cryostat-0001/6852D/BOT_acq/v0/48113/ccob_nm960_-179.195_-52.805_018/MC_C_20191028_005285_R12_S00.fits']


../ccob_qe_data.py:86: ResourceWarning: unclosed file <_io.FileIO name='/home/combet/tmp_9rafts/tmp.fits' mode='rb' closefd=True>
  mosaic, amp_coord = u.make_ccd_2d_array(tmp_file, gains=self.gains, biasfile=self.biasfile)


23.629698996655517 34.97983277591973 17.330301003344484 5.060167224080269
40.96 40.04
(4004, 4096)
40.04
40.96


In [5]:
data.dir_list

array(['/gpfs/slac/lsst/fs3/g/data/jobHarness/jh_stage-test/LCA-10134_Cryostat/LCA-10134_Cryostat-0001/6852D/BOT_acq/v0/48113/ccob_nm960_-159.175_-32.325_006',
       '/gpfs/slac/lsst/fs3/g/data/jobHarness/jh_stage-test/LCA-10134_Cryostat/LCA-10134_Cryostat-0001/6852D/BOT_acq/v0/48113/ccob_nm960_-159.175_-52.805_000',
       '/gpfs/slac/lsst/fs3/g/data/jobHarness/jh_stage-test/LCA-10134_Cryostat/LCA-10134_Cryostat-0001/6852D/BOT_acq/v0/48113/ccob_nm960_-169.185_-42.565_012',
       '/gpfs/slac/lsst/fs3/g/data/jobHarness/jh_stage-test/LCA-10134_Cryostat/LCA-10134_Cryostat-0001/6852D/BOT_acq/v0/48113/ccob_nm960_-179.195_-32.325_024',
       '/gpfs/slac/lsst/fs3/g/data/jobHarness/jh_stage-test/LCA-10134_Cryostat/LCA-10134_Cryostat-0001/6852D/BOT_acq/v0/48113/ccob_nm960_-179.195_-52.805_018'],
      dtype='<U149')

In [ ]:
import lsst.eotest.sensor as sensorTest
import lsst.eotest.raft as eotest_raft
import lsst.eotest.image_utils as imutils
from lsst.eotest.sensor.EOTestPlots import cmap_range
import numpy as np
import glob
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
def plot(dd, title=None, cmap=plt.cm.hot, nsig=5, figsize=(10, 10),
             binsize=10, flipx=True, textcolor='c', annotation='',
             rotate180=False):
        """
        Render the raft mosaic.
        Parameters
        ----------
        title : str, optional
            The plot title. If None (default), then build the title
            from the RAFTNAME and MONOWL primary header keyword values.
        cmap : matplotlib.colors.Colormap, optional
            The color map to use. Default: matplotlib.pyplot.cm.hot.
        nsig : float, optional
            The n-sigma value for the sigma clipping used to determine
            the pixel value range over which the color map is mapped.
        figsize : (float, float), optional
            The width x height size of the figure in inches. Default: (10, 10).
        binsize : int, optional
            Rebin the plotted image data by binsize*binsize,
            averging over the coarser bin.  Default: 10
        flipx : bool, optional
            Flip full raft mosaic in x so that parity of image matches
            LCA-13381. Default: True
        textcolor : str, optional
            Color of the text for the segment and sensor labeling.
            Default: 'c' (cyan)
        annotation : str, optional
            Description of the plot, e.g., pixel units (ADU or e-),
            gain-corrected, bias-subtracted.  Default: ''
        rotate180 : bool [False]
            Flag to rotate the mosaic by 180 degrees to match the
            orientation of the focalplane mosiacs created for the
            BOT-level plots.
        """
        plt.rcParams['figure.figsize'] = figsize
        fig = plt.figure()
        ax = fig.add_subplot(1, 1, 1)
        output_array = imutils.rebin_array(dd.image_array, binsize,
                                           use_mean=True)
        if flipx:
            output_array = output_array[:, ::-1]
        if rotate180:
            ny, nx = output_array.shape
            rotated_array = np.zeros((nx, ny), dtype=output_array.dtype)
            for j in range(ny):
                rotated_array[:, ny-1-j] = output_array[::-1, j]
            output_array = rotated_array
        image = ax.imshow(output_array, interpolation='nearest', cmap=cmap)
        # Set range and normalization of color map based on sigma-clip
        # of pixel values.
        vmin, vmax = cmap_range(output_array, nsig=nsig)
        norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
        image.set_norm(norm)
        if title is None:
            title = "%s, %i nm" % (dd.raft_name, dd.wl)
        ax.set_title(title)
        fig.colorbar(image)
        # Turn off ticks and tick labels for x- and y-axes.
        plt.tick_params(axis='both', which='both',
                        top='off', bottom='off', left='off', right='off',
                        labelbottom='off', labelleft='off')
        # Label segments by sensor bay and segment number.
        for slot in dd.fits_files:
            seg_coords = list(dd._amp_coords[slot].values())[-8]
            xmin, xmax, ymin, ymax = seg_coords
            xx = float(xmax + xmin)/2./float(dd.nx)
            if flipx:
                xx = 1 - xx
            yy = 1. - (float(ymax - ymin)*0.05 + ymin)/float(dd.ny)
            if rotate180:
                xx = 1 - xx - 7*np.abs(xmax - xmin)/float(dd.nx)
                yy = 1 - yy + 1.9*np.abs(ymax - ymin)/float(dd.ny)
            plt.annotate('%s' % slot,
                         (xx, yy), xycoords='axes fraction',
                         size='x-small', horizontalalignment='center',
                         verticalalignment='center', color=textcolor)
            for amp, seg_coords in list(dd._amp_coords[slot].items()):
                xmin, xmax, ymin, ymax = seg_coords
                xx = float(xmax + xmin)/2./float(dd.nx)
                if flipx:
                    xx = 1. - xx
                if amp <= 8:
                    yy = 1. - (float(ymax - ymin)*0.85 + ymin)/float(dd.ny)
                else:
                    yy = 1. - (float(ymax - ymin)*0.15 + ymin)/float(dd.ny)
                if rotate180:
                    xx = 1 - xx
                    yy = 1 - yy
                plt.annotate('%s' % imutils.channelIds[amp],
                             (xx, yy), xycoords='axes fraction',
                             size='x-small', horizontalalignment='center',
                             verticalalignment='center', color=textcolor)
        plt.annotate(annotation, (1, -0.1), xycoords='axes fraction',
                     horizontalalignment='right', verticalalignment='bottom')
        return fig

In [ ]:
raft = 'R22'
slot_names=['S00','S01','S02','S10','S11','S12','S20','S21','S22']
led = 'blue'
#slot = slot_names[0]
#sorted(glob.glob(f'/gpfs/slac/lsst/fs1/u/combet/DATA/CCOB_QE/fits/QE_{raft}_{slot}_{led}*'))[2]
#[(glob.glob(f'/gpfs/slac/lsst/fs1/u/combet/DATA/CCOB_QE/fits/QE_{raft}_{slot}_{led}*')[2])[0] for slot in slot_names]
fits_files_dict ={slot : sorted(glob.glob(f'/gpfs/slac/lsst/fs1/u/combet/DATA/CCOB_QE/fits/QE_{raft}_{slot}_{led}*'))[2] for slot in slot_names}
fits_files_dict

In [ ]:
dd = eotest_raft.RaftMosaic(fits_files_dict)

In [ ]:
raft_name = dd.raft_name.split('_')[1]

In [ ]:
tmp = plot(dd,rotate180=True, nsig=4, title=f'CCOB composite flat - {led} - {raft} ({raft_name})')

In [ ]:
outdir_figs = '/home/combet/tmp_9rafts/QE_results/figs/'
tmp.savefig(outdir_figs+f'{raft}_flat_{led}.png')

In [6]:
from astropy.io import fits as fits

In [7]:
hdu = fits.open('/gpfs/slac/lsst/fs1/u/combet/DATA/CCOB_QE/fits/QE_R02_S00_blue_-286.285_-32.685.fits')

In [12]:
hdu[0].header

SIMPLE  =                    T / Java FITS: Mon Oct 28 17:54:09 PDT 2019        
BITPIX  =                    8 / bits per data value                            
NAXIS   =                    0 / number of axes                                 
EXTEND  =                    T / Extensions are permitted                       
DATE    = '2019-10-29T02:27:48.545' / Creation Date and Time of File            
DATE-OBS= '2019-10-29T02:27:46.151' / Date of the image trigger (readout), UTC f
MJD     =    58785.10264519695 / Modified Julian Date that the file was written 
MJD-OBS =    58785.10261748824 / Modified Julian Date of image trigger          
RUNNUM  = '6852D   '           / The Run Number                                 
IMAGETAG= 'd9b96a33ead3fba4'   / DAQ Image id (Hex)                             
TSTAND  = 'BOT     '           / Camera test stand BOT or CCOB                  
INSTRUME= 'lsstCam '           / Instrument                                     
CONTNUM = '18ee34d8'        

In [3]:
import ccob_utils as u
f1 = '/gpfs/slac/lsst/fs1/u/combet/DATA/CCOB_QE/fits/QE_R22_S11_red_0.005_0.005.fits'
im, dum = u.make_ccd_2d_array(f1)

In [ ]:
f2 = '/gpfs/slac/lsst/fs1/u/combet/DATA/CCOB_QE/fits/QE_R22_S11_red_10.015_-10.235.fits'
im2, dum = u.make_ccd_2d_array(f2)

In [ ]:
import scipy
#residuals = (im-im2)*100/im
residuals = scipy.ndimage.filters.gaussian_filter((im-im2)/im, 1, mode='constant')
plt.imshow(residuals, vmin=-0.002, vmax=0.002)
plt.colorbar()

In [9]:
f = '/gpfs/slac/lsst/fs3/g/data/jobHarness/jh_stage-test/LCA-10134_Cryostat/LCA-10134_Cryostat-0001/6852D/BOT_acq/v0/48113/ccob_nm960_-159.175_-32.325_006/MC_C_20191028_005273_R12_S00.fits'

hdu = fits.open(f)

In [10]:
hdu[0].header

SIMPLE  =                    T / Java FITS: Mon Oct 28 17:54:09 PDT 2019        
BITPIX  =                    8 / bits per data value                            
NAXIS   =                    0 / number of axes                                 
EXTEND  =                    T / Extensions are permitted                       
DATE    = '2019-10-29T02:27:48.545' / Creation Date and Time of File            
DATE-OBS= '2019-10-29T02:27:46.151' / Date of the image trigger (readout), UTC f
MJD     =    58785.10264519695 / Modified Julian Date that the file was written 
MJD-OBS =    58785.10261748824 / Modified Julian Date of image trigger          
RUNNUM  = '6852D   '           / The Run Number                                 
IMAGETAG= 'd9b96a33ead3fba4'   / DAQ Image id (Hex)                             
TSTAND  = 'BOT     '           / Camera test stand BOT or CCOB                  
INSTRUME= 'lsstCam '           / Instrument                                     
CONTNUM = '18ee34d8'        